In [7]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import requests

In [5]:
params = {
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

data=params

In [8]:
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_PRICES
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
items_id = fetch_items_from_storage(storage, settings.BUCKET_STORES, blob_items_prefix, 'id')
print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving price details
blob_basic_path = settings.BLOB_PRICES(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_PRICE
url

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=0__total_items=100__process_time=2024-09-23T10:15:15.156411-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=1__total_items=100__process_time=2024-09-23T10:15:15.748949-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=2__total_items=100__process_time=2024-09-23T10:15:15.397569-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=3__total_items=100__process_time=2024-09-23T10:15:15.991552-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=4__

<function src.config.settings.<lambda>(item_id)>

In [16]:
headers = {'Authorization': f'Bearer {access_token}'}

In [47]:
response = requests.get(url('MLB3348592687'), headers=headers)
json = response.json()
json

{'id': 'MLB3348592687',
 'prices': [{'id': '26',
   'type': 'standard',
   'amount': 369,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2023-11-09T13:54:49Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '243',
   'type': 'promotion',
   'amount': 287.82,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': None,
    'end_time': None}},
  {'id': '244',
   'type': 'promotion',
   'amount': 287.82,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': None,
    'end_time': None}},
  {'id': '245',
   'type': 'promotion',
   'amount': 284.13,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'c

In [48]:
extracted_data = []
# Dicionário temporário para priorizar os preços por canal
price_by_channel = {}
for price in json['prices']:
    channel = price['conditions']['context_restrictions']
    if len(channel) == 1:
        channel = channel[0]
        # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
        if channel not in price_by_channel or price['type'] == 'promotion':
            price_by_channel[channel] = {
                'item_id': json.get('id'),
                'price_id': price.get('id'),
                'regular_amount': price.get('regular_amount'),
                'price': price.get('amount'),
                'channel': channel,
                'last_updated': price.get('last_updated')
            }
    else:
        channel = None
        if price_by_channel=={}:
            price_by_channel[channel] = {
                    'item_id': json.get('id'),
                    'price_id': price.get('id'),
                    'regular_amount': price.get('regular_amount'),
                    'price': price.get('amount'),
                    'channel': channel,
                    'last_updated': price.get('last_updated')
                }

# Converte os valores armazenados para uma lista
extracted_data.extend(price_by_channel.values())
extracted_data

[{'item_id': 'MLB3348592687',
  'price_id': '26',
  'regular_amount': None,
  'price': 369,
  'channel': None,
  'last_updated': '2023-11-09T13:54:49Z'},
 {'item_id': 'MLB3348592687',
  'price_id': '244',
  'regular_amount': 369.0,
  'price': 287.82,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-16T12:41:24Z'},
 {'item_id': 'MLB3348592687',
  'price_id': '245',
  'regular_amount': 369.0,
  'price': 284.13,
  'channel': 'channel_mshops',
  'last_updated': '2024-09-16T12:41:24Z'}]

In [26]:
price_by_channel == {}

True